### Questão 5: Alterar a arquitetura ResNet "convolution, batch normalization, ReLu" para "batch normalization, ReLu, convolution". Comparar se há diferença no desempenho da rede.

In [ ]:
 !pip install d2l==1.0.0-alpha1.post0 

In [ ]:
import tensorflow as tf
from d2l import tensorflow as d2l
from keras.datasets import cifar10
import numpy as np

device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

def convolutional_block(x, filter):
    
    # copy tensor to variable called x_skip
    x_skip = x
    
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    
    return x

def identity_block(x, filter):
    
    # copy tensor to variable called x_skip
    x_skip = x
    
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    
    return x

Found GPU at: /device:GPU:0


In [ ]:
def ResNet34(shape = (32, 32, 3), classes = 10):
  
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
          
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [ ]:
# load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# summarize loaded dataset
print(f'Train: X= {x_train.shape}, y= {y_train.shape}')
print(f'Test: X={x_test.shape}, y={y_test.shape}')

print(x_train.dtype, x_test.dtype)
# Normalizing data 
# As the Data is of type uint8 we will convert it to Float
x_train = x_train.astype(float)
x_test = x_test.astype(float)
x_train = x_train/255.0
x_test = x_test/255.0

# Shuffle first (optional)
idx = np.arange(len(x_train))
np.random.shuffle(idx)

# get first 10% of data
x_train = x_train[:int(.10*len(idx))]
y_train = y_train[:int(.10*len(idx))]
x_test = x_test[:int(.10*len(idx))]
y_test = y_test[:int(.10*len(idx))]

Train: X= (50000, 32, 32, 3), y= (50000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)
uint8 uint8


In [ ]:
model = ResNet34()
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train, validation_data=(x_test,y_test), epochs=5)


Epoch 1/5
157/157 [==============================] - 12s 54ms/step - loss: 2.1409 - accuracy: 0.2384 - val_loss: 3.8857 - val_accuracy: 0.1690
Epoch 2/5
157/157 [==============================] - 8s 53ms/step - loss: 1.7513 - accuracy: 0.3506 - val_loss: 1.7917 - val_accuracy: 0.3106
Epoch 3/5
157/157 [==============================] - 8s 50ms/step - loss: 1.6216 - accuracy: 0.4072 - val_loss: 2.3396 - val_accuracy: 0.3070
Epoch 4/5
157/157 [==============================] - 11s 68ms/step - loss: 1.4819 - accuracy: 0.4608 - val_loss: 1.6203 - val_accuracy: 0.4430
Epoch 5/5
157/157 [==============================] - 9s 55ms/step - loss: 1.4000 - accuracy: 0.4958 - val_loss: 1.8433 - val_accuracy: 0.3670


In [ ]:
def ResNetModified(shape = (32, 32, 3), classes = 10):
  
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
          
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [ ]:
model = ResNetModified()
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

Epoch 1/5
157/157 [==============================] - 12s 54ms/step - loss: 2.1916 - accuracy: 0.2200 - val_loss: 2.3757 - val_accuracy: 0.1774
Epoch 2/5
157/157 [==============================] - 8s 54ms/step - loss: 1.8630 - accuracy: 0.3024 - val_loss: 1.9792 - val_accuracy: 0.2788
Epoch 3/5
157/157 [==============================] - 8s 49ms/step - loss: 1.7269 - accuracy: 0.3502 - val_loss: 3.2479 - val_accuracy: 0.2224
Epoch 4/5
157/157 [==============================] - 8s 50ms/step - loss: 1.6298 - accuracy: 0.3930 - val_loss: 2.0110 - val_accuracy: 0.3338
Epoch 5/5
157/157 [==============================] - 8s 50ms/step - loss: 1.5535 - accuracy: 0.4320 - val_loss: 1.9599 - val_accuracy: 0.3656


### A principio não dá para se ver uma diferença significativa no desempenho da rede, mas é possível que com mais épocas de treinamento a diferença seja mais significativa.